In [1]:
import pandas as pd
import numpy as np

In [2]:
posteriors_5_npEM = pd.read_csv('./results/np_posteriors/npEM_k_5_1_5_CPMcutoff_suffix_1_log_cero_replacement_posteriors.csv', index_col = 0)
posteriors_5_npMSL = pd.read_csv('./results/np_posteriors/npMSL_k_5_1_5_CPMcutoff_suffix_1_log_cero_replacement_posteriors.csv', index_col = 0)
posteriors_8_npEM = pd.read_csv('./results/np_posteriors/npEM_k_8_1_5_CPMcutoff_suffix_1_log_cero_replacement_posteriors.csv', index_col = 0)
posteriors_8_npMSL = pd.read_csv('./results/np_posteriors/npMSL_k_8_1_5_CPMcutoff_suffix_1_log_cero_replacement_posteriors.csv', index_col = 0)

In [3]:
names = ['npEM_5','npMSL_5','npEM_8','npMSL_8']
datasets = [posteriors_5_npEM,posteriors_5_npMSL,posteriors_8_npEM,posteriors_8_npMSL]

In [14]:
df = pd.DataFrame({})
for name, data in zip (names,datasets):
    df[name] = np.argmax(data.values, axis = 1)
df.head()

,npEM_5,npMSL_5,npEM_8,npMSL_8
0,1,4,5,3
1,1,0,6,4
2,1,4,5,3
3,3,4,7,3
4,1,0,6,4


In [15]:
df.to_csv('./results/np_labels/grouped_labels.csv')